In [1]:
import glob
import os

data_path = 'UCF101_frames'  # your extracted frames root folder

class_folders = glob.glob(os.path.join(data_path, '*'))  # class folders
print(f'Number of classes found: {len(class_folders)}')

total_pngs = 0
for cls in class_folders:
    video_folders = glob.glob(os.path.join(cls, '*'))  # video folders inside class
    print(f'Class: {cls}, Number of videos: {len(video_folders)}')
    for vid in video_folders:
        png_files = glob.glob(os.path.join(vid, '*.png'))
        print(f'Video: {vid}, PNG count: {len(png_files)}')
        total_pngs += len(png_files)

print(f'Total PNG frames found: {total_pngs}')


Number of classes found: 15
Class: UCF101_frames\ApplyEyeMakeup, Number of videos: 101
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c01, PNG count: 120
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c02, PNG count: 117
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c03, PNG count: 146
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c04, PNG count: 224
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c05, PNG count: 276
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c01, PNG count: 176
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c02, PNG count: 258
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c03, PNG count: 210
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c04, PNG count: 191
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c05, PNG count: 194
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c06, PNG count: 188
Video: UCF101_frames\ApplyEyeMakeup\v_ApplyEyeMakeup_g09_c07, PNG 

In [30]:
import os
import tensorflow as tf
import cv2
import numpy as np
from datetime import datetime
from vgg16_keras import VGG16FeatureExtractor 
from CyclicGen_model import Voxel_flow_model
# ==== YOUR 15 TARGET CLASSES ====
TARGET_CLASSES = [
    'ApplyEyeMakeup', 'Archery', 'BabyCrawling', 'Basketball', 'BenchPress',
    'Biking', 'Bowling', 'BoxingPunchingBag', 'CliffDiving', 'Diving',
    'Drumming', 'GolfSwing', 'Haircut', 'HorseRiding', 'JumpRope'
]

In [4]:
import os
import glob

def generate_triplets(data_dir, output_file):
    triplets = []
    classes = sorted(os.listdir(data_dir))
    for cls in classes:
        cls_path = os.path.join(data_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        videos = sorted(os.listdir(cls_path))
        for vid in videos:
            vid_path = os.path.join(cls_path, vid)
            if not os.path.isdir(vid_path):
                continue
            frames = sorted(glob.glob(os.path.join(vid_path, '*')))
            for i in range(len(frames) - 2):
                triplet = f"{frames[i]} {frames[i+1]} {frames[i+2]}"
                triplets.append(triplet)
    with open(output_file, 'w') as f:
        for triplet in triplets:
            f.write(f"{triplet}\n")
    print(f"Generated {len(triplets)} triplets.")

# Usage
generate_triplets('D:/VFI/CyclicGen/UCF101_frames', 'triplets.txt')



Generated 296210 triplets.


In [5]:
import glob
frames = glob.glob("D:/VFI/CyclicGen/UCF101_frames/**/*.png", recursive=True)
print(f"Found {len(frames)} frames")


Found 299356 frames


In [31]:
def read_triplets_from_txt(txt_path):
    f1_list, f2_list, f3_list = [], [], []
    with open(txt_path, 'r') as file:
        for line in file:
            f1, f2, f3 = line.strip().split()
            f1_list.append(f1)
            f2_list.append(f2)
            f3_list.append(f3)
    return f1_list, f2_list, f3_list

In [32]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [36]:

def create_dataset(file_list, batch_size):
    path_ds = tf.data.Dataset.from_tensor_slices(file_list)
    image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size).repeat()
    return iter(image_ds)  # Use Python iterator for TF 2.x


In [12]:
def get_vgg16_edge_features(input_tensor):
    base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    edge_feature = base_model.get_layer('block5_conv3').output
    edge_feature = tf.nn.sigmoid(edge_feature)
    edge_feature = tf.image.resize(edge_feature, (256, 256))
    edge_feature = tf.reduce_mean(edge_feature, axis=-1, keepdims=True)
    return edge_feature


In [39]:
from CyclicGen_model import Voxel_flow_model
def train_epochwise_using_triplets(triplets_txt_path, batch_size, num_epochs, train_dir):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    f1_list, f2_list, f3_list = read_triplets_from_txt(triplets_txt_path)
    dataset_size = len(f1_list)
    steps_per_epoch = dataset_size // batch_size

    it1 = iter(create_dataset(f1_list, batch_size))
    it2 = iter(create_dataset(f2_list, batch_size))
    it3 = iter(create_dataset(f3_list, batch_size))

# Use eager tensors
    input1 = next(it1)
    input2 = next(it2)
    input3 = next(it3)

# Apply VGG and model in eager mode
    vgg_model = VGG16FeatureExtractor()
    edge_1 = vgg_model(input1)
    edge_3 = vgg_model(input3)

# Call model
    prediction = Voxel_flow_model(tf.concat([input1, input3, edge_1, edge_3], axis=3))

# Calculate loss
    loss = tf.reduce_mean(tf.abs(prediction - input2))

    with tf.variable_scope("Cycle_DVF"):
        model = Voxel_flow_model()
        concat_input = tf.concat([input1, input3, edge_1, edge_3], axis=3)
        prediction, flow = model.inference(concat_input)
        loss = model.l1loss(prediction, input2)

    trainable_vars = tf.trainable_variables()
    dof_vars = [v for v in trainable_vars if not 'vgg' in v.name]
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss, var_list=dof_vars)

    saver = tf.train.Saver(tf.global_variables(), max_to_keep=50)
    summary_op = tf.summary.merge([
            tf.summary.scalar('loss', loss),
            tf.summary.image('input1', input1, 1),
            tf.summary.image('input2', input2, 1),
            tf.summary.image('input3', input3, 1),
            tf.summary.image('predicted', prediction, 1),
        ])

    loss_history = []

    with tf.Session(graph=g) as sess:
            sess.run(tf.global_variables_initializer())
            sess.run([it1.initializer, it2.initializer, it3.initializer])
            summary_writer = tf.summary.FileWriter(train_dir, sess.graph)

            for epoch in range(num_epochs):
                print(f"\nEpoch {epoch + 1}/{num_epochs}")
                total_loss = 0

                for step in range(steps_per_epoch):
                    _, loss_val = sess.run([optimizer, loss])
                    total_loss += loss_val

                    if step % 100 == 0:
                        summary_writer.add_summary(sess.run(summary_op), epoch * steps_per_epoch + step)

                avg_loss = total_loss / steps_per_epoch
                loss_history.append(avg_loss)
                print(f"Avg Loss: {avg_loss:.5f}")

                saver.save(sess, os.path.join(train_dir, "model.ckpt"), global_step=epoch)

            print(" Training complete. Saving loss plot...")

            plt.figure()
            plt.plot(range(1, num_epochs + 1), loss_history, marker='o')
            plt.xlabel("Epoch")
            plt.ylabel("Average Loss")
            plt.title("Training Loss per Epoch")
            plt.grid(True)
            plt.savefig(os.path.join(train_dir, "loss_curve.png"))
            plt.show()


In [40]:
train_epochwise_using_triplets(
    triplets_txt_path="D:/VFI/CyclicGen/triplets.txt",
    batch_size=4,
    num_epochs=10,
    train_dir="training_logs"
)

RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.